In [1]:
total_value=int(input("Enter the total money(in Numerical form)"))

Enter the total money(in Numerical form)10000000


In [2]:
Num_Companies=int(input("Enter the total money(in Numerical form)"))

Enter the total money(in Numerical form)45


In [3]:
import datapackage
import pandas as pd

data_url = 'https://datahub.io/core/s-and-p-500-companies/datapackage.json'

# to load Data Package into storage
package = datapackage.Package(data_url)

# to load only tabular data
resources = package.resources
for resource in resources:
    if resource.tabular:
        dataset = pd.read_csv(resource.descriptor['path'])
sym_name=dict()
for i in range(len(dataset["Symbol"])):
    sym_name[dataset["Symbol"][i]]=dataset["Name"][i]
dataset

,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care
...,...,...,...
500,YUM,Yum! Brands,Consumer Discretionary
501,ZBRA,Zebra Technologies,Information Technology
502,ZBH,Zimmer Biomet,Health Care
503,ZION,Zions Bancorp,Financials


In [4]:
def splitter(symbols,size):
    li=list()
    start=0
    while start<len(symbols):
        x=""
        if start+size<len(symbols):
            end=start+size
        else:
            end=len(symbols)
        for i in range(start,end):
            x+=(symbols[i]+",")
        x=x[:-1]
        li.append(x)
        start+=size
    return li

In [5]:
import numpy as np
import requests
import time 
#please enter your generated token key in token platform 
token=""
symbols=dataset["Symbol"]
splits=splitter(symbols,80)

In [35]:
cols = ['Company',
    'Symbol',
    'Price',
    'Quantity', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

In [44]:
splits=splitter(symbols,80)
li=list()
#Please enter your token generated
token=""
for sym in splits:
    #in case you are using actual version instead of sandbox please remove sandbox from url
    url='https://sandbox.iexapis.com/stable/stock/market/batch?symbols='+sym+'&types=quote,advanced-stats&token='+token
    data=requests.get(url).json()
    for i in data:
        enterprise_value = data[i]['advanced-stats']['enterpriseValue']
        ebitda = data[i]['advanced-stats']['EBITDA']
        gross_profit = data[i]['advanced-stats']['grossProfit']        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        
        li.append([sym_name[i],i,data[i]['quote']['latestPrice'],'N/A',data[i]['quote']['peRatio'],'N/A',data[i]['advanced-stats']['priceToBook'],'N/A',data[i]['advanced-stats']['priceToSales'],'N/A',ev_to_ebitda,'N/A',ev_to_gross_profit,'N/A','N/A'])

In [45]:
stockset=pd.DataFrame(li,columns=cols)
stockset

,Company,Symbol,Price,Quantity,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,3M,MMM,171.982,N/A,16.83,N/A,6.79,N/A,2.73,N/A,11.266497,N/A,6.393406,N/A,N/A
1,A. O. Smith,AOS,76.350,N/A,27.00,N/A,6.71,N/A,3.74,N/A,17.502096,N/A,8.977101,N/A,N/A
2,Abbott Laboratories,ABT,128.300,N/A,31.90,N/A,6.64,N/A,5.33,N/A,22.009647,N/A,9.823740,N/A,N/A
3,AbbVie,ABBV,138.340,N/A,32.91,N/A,18.69,N/A,4.45,N/A,11.832145,N/A,8.665871,N/A,N/A
4,Abiomed,ABMD,300.685,N/A,90.95,N/A,9.54,N/A,13.09,N/A,42.924390,N/A,15.548913,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,Yum! Brands,YUM,128.700,N/A,24.60,N/A,-4.68,N/A,5.70,N/A,20.663418,N/A,9.779363,N/A,N/A
500,Zebra Technologies,ZBRA,525.220,N/A,33.43,N/A,9.80,N/A,4.84,N/A,23.770221,N/A,10.228822,N/A,N/A
501,Zimmer Biomet,ZBH,124.020,N/A,32.10,N/A,2.05,N/A,3.30,N/A,17.500083,N/A,5.791218,N/A,N/A
502,Zions Bancorp,ZION,68.628,N/A,6.61,N/A,1.49,N/A,3.09,N/A,5.248899,N/A,2.849621,N/A,N/A


In [46]:
nullcols=['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']
for i in nullcols:
    stockset[i].fillna(stockset[i].mean(), inplace = True)

In [47]:
from scipy import stats
for i in range(len(stockset["Symbol"])):
    stockset.loc[i, 'PE Percentile'] = stats.percentileofscore(stockset['Price-to-Earnings Ratio'], stockset.loc[i, 'Price-to-Earnings Ratio'])
    stockset.loc[i, 'PB Percentile'] = stats.percentileofscore(stockset['Price-to-Book Ratio'], stockset.loc[i, 'Price-to-Book Ratio'])
    stockset.loc[i,'PS Percentile' ] = stats.percentileofscore(stockset['Price-to-Sales Ratio'], stockset.loc[i, 'Price-to-Sales Ratio'])
    stockset.loc[i, 'EV/EBITDA Percentile'] = stats.percentileofscore(stockset['EV/EBITDA'], stockset.loc[i, 'EV/EBITDA'])
    stockset.loc[i,'EV/GP Percentile' ] = stats.percentileofscore(stockset['EV/GP'], stockset.loc[i, 'EV/GP'])
stockset

,Company,Symbol,Price,Quantity,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,3M,MMM,171.982,N/A,16.83,32.5397,6.79,74.0079,2.73,43.4524,11.266497,36.7063,6.393406,43.8492,N/A
1,A. O. Smith,AOS,76.350,N/A,27.00,56.3492,6.71,73.4127,3.74,56.8452,17.502096,59.7222,8.977101,65.6746,N/A
2,Abbott Laboratories,ABT,128.300,N/A,31.90,65.0794,6.64,72.9167,5.33,72.8175,22.009647,74.2063,9.823740,69.246,N/A
3,AbbVie,ABBV,138.340,N/A,32.91,67.6587,18.69,92.4603,4.45,63.4921,11.832145,38.4921,8.665871,63.2937,N/A
4,Abiomed,ABMD,300.685,N/A,90.95,95.4365,9.54,81.746,13.09,92.6587,42.924390,95.2381,15.548913,87.5,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,Yum! Brands,YUM,128.700,N/A,24.60,52.1825,-4.68,5.35714,5.70,76.3889,20.663418,70.8333,9.779363,68.8492,N/A
500,Zebra Technologies,ZBRA,525.220,N/A,33.43,69.1468,9.80,82.5397,4.84,67.4603,23.770221,79.3651,10.228822,71.627,N/A
501,Zimmer Biomet,ZBH,124.020,N/A,32.10,65.4762,2.05,29.7619,3.30,51.0913,17.500083,59.5238,5.791218,39.2857,N/A
502,Zions Bancorp,ZION,68.628,N/A,6.61,10.9127,1.49,17.3611,3.09,48.0159,5.248899,7.7381,2.849621,14.6825,N/A


In [48]:
from statistics import mean
li=list()
for i in range(len(stockset["Symbol"])):
    li.append(stockset.loc[i,'PE Percentile'])
    li.append(stockset.loc[i,'PB Percentile'])
    li.append(stockset.loc[i,'PS Percentile'])
    li.append(stockset.loc[i,'EV/EBITDA Percentile'])
    li.append(stockset.loc[i,'EV/GP Percentile'])
    stockset.loc[i, 'RV Score'] = mean(li)
stockset
    
    

,Company,Symbol,Price,Quantity,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,3M,MMM,171.982,N/A,16.83,32.5397,6.79,74.0079,2.73,43.4524,11.266497,36.7063,6.393406,43.8492,46.1111
1,A. O. Smith,AOS,76.350,N/A,27.00,56.3492,6.71,73.4127,3.74,56.8452,17.502096,59.7222,8.977101,65.6746,54.256
2,Abbott Laboratories,ABT,128.300,N/A,31.90,65.0794,6.64,72.9167,5.33,72.8175,22.009647,74.2063,9.823740,69.246,59.7884
3,AbbVie,ABBV,138.340,N/A,32.91,67.6587,18.69,92.4603,4.45,63.4921,11.832145,38.4921,8.665871,63.2937,61.1111
4,Abiomed,ABMD,300.685,N/A,90.95,95.4365,9.54,81.746,13.09,92.6587,42.924390,95.2381,15.548913,87.5,66.9921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,Yum! Brands,YUM,128.700,N/A,24.60,52.1825,-4.68,5.35714,5.70,76.3889,20.663418,70.8333,9.779363,68.8492,50.0336
500,Zebra Technologies,ZBRA,525.220,N/A,33.43,69.1468,9.80,82.5397,4.84,67.4603,23.770221,79.3651,10.228822,71.627,50.0815
501,Zimmer Biomet,ZBH,124.020,N/A,32.10,65.4762,2.05,29.7619,3.30,51.0913,17.500083,59.5238,5.791218,39.2857,50.0794
502,Zions Bancorp,ZION,68.628,N/A,6.61,10.9127,1.49,17.3611,3.09,48.0159,5.248899,7.7381,2.849621,14.6825,50.0191


In [49]:
stockset.sort_values(by = 'RV Score', inplace = True)
stockset = stockset[:Num_Companies]
stockset.reset_index(drop = True, inplace = True)
stockset

,Company,Symbol,Price,Quantity,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,3M,MMM,171.982,N/A,16.83,32.5397,6.7900,74.0079,2.7300,43.4524,11.266497,36.7063,6.393406,43.8492,46.1111
1,Raymond James Financial,RJF,109.010,N/A,8.09,13.6905,1.3400,13.8889,1.1300,15.0794,5.304403,8.13492,1.085565,3.37302,49.4949
2,Raytheon Technologies,RTX,92.088,N/A,41.50,79.1667,1.9000,26.1905,2.4800,40.2778,21.496011,73.0159,17.648774,90.6746,49.5264
3,Regions Financial Corporation,RF,22.940,N/A,9.00,15.2778,1.4000,15.4762,3.4000,52.6786,5.815476,10.7143,2.987977,15.873,49.5479
4,Marathon Petroleum,MPC,72.740,N/A,5.06,8.53175,1.6100,21.7262,0.4762,3.96825,10.422036,31.9444,8.442213,60.7143,49.5666
5,Republic Services,RSG,133.035,N/A,34.80,71.4286,4.7200,64.0873,3.8100,57.6389,16.005316,56.1508,11.355394,76.1905,49.5871
6,Ralph Lauren Corporation,RL,115.250,N/A,20.48,41.8651,2.9100,43.1548,1.4400,21.8254,7.255774,15.2778,1.851989,9.3254,49.594
7,Realty Income Corporation,O,72.710,N/A,55.06,86.7063,2.1900,31.746,16.0800,96.4286,23.121049,77.7778,20.083488,93.8492,49.5971
8,Regeneron Pharmaceuticals,REGN,636.834,N/A,9.97,17.4603,3.8500,52.381,4.8900,68.0556,8.318385,22.2222,4.958457,31.3492,49.6176
9,Regency Centers,REG,74.560,N/A,38.53,76.5873,2.0300,29.5635,10.9900,89.881,19.205335,65.873,13.611741,83.1349,49.6464


In [50]:
for i in range(len(stockset["Price"])):
    stockset['Quantity'][i]=total_value//(stockset['Price'][i]*Num_Companies)
stockset
    

<ipython-input-50-04f1a33b265b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stockset['Quantity'][i]=total_value//(stockset['Price'][i]*Num_Companies)
C:\Users\Akshay\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,Company,Symbol,Price,Quantity,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,3M,MMM,171.982,1292,16.83,32.5397,6.7900,74.0079,2.7300,43.4524,11.266497,36.7063,6.393406,43.8492,46.1111
1,Raymond James Financial,RJF,109.010,2038,8.09,13.6905,1.3400,13.8889,1.1300,15.0794,5.304403,8.13492,1.085565,3.37302,49.4949
2,Raytheon Technologies,RTX,92.088,2413,41.50,79.1667,1.9000,26.1905,2.4800,40.2778,21.496011,73.0159,17.648774,90.6746,49.5264
3,Regions Financial Corporation,RF,22.940,9687,9.00,15.2778,1.4000,15.4762,3.4000,52.6786,5.815476,10.7143,2.987977,15.873,49.5479
4,Marathon Petroleum,MPC,72.740,3055,5.06,8.53175,1.6100,21.7262,0.4762,3.96825,10.422036,31.9444,8.442213,60.7143,49.5666
5,Republic Services,RSG,133.035,1670,34.80,71.4286,4.7200,64.0873,3.8100,57.6389,16.005316,56.1508,11.355394,76.1905,49.5871
6,Ralph Lauren Corporation,RL,115.250,1928,20.48,41.8651,2.9100,43.1548,1.4400,21.8254,7.255774,15.2778,1.851989,9.3254,49.594
7,Realty Income Corporation,O,72.710,3056,55.06,86.7063,2.1900,31.746,16.0800,96.4286,23.121049,77.7778,20.083488,93.8492,49.5971
8,Regeneron Pharmaceuticals,REGN,636.834,348,9.97,17.4603,3.8500,52.381,4.8900,68.0556,8.318385,22.2222,4.958457,31.3492,49.6176
9,Regency Centers,REG,74.560,2980,38.53,76.5873,2.0300,29.5635,10.9900,89.881,19.205335,65.873,13.611741,83.1349,49.6464


In [52]:
import xlsxwriter
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
stockset.to_excel(writer, sheet_name='Value Strategy', index = False)


In [53]:
background_color = '#ffffff'
font_color = '#000000'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [55]:
column_formats = {
                    'A': ['Company',string_template],
                    'B': ['Symbol', string_template],
                    'C': ['Price', dollar_template],
                    'D': ['Quantity', integer_template],
                    'E': ['Price-to-Earnings Ratio', float_template],
                    'F': ['PE Percentile', percent_template],
                    'G': ['Price-to-Book Ratio', float_template],
                    'H': ['PB Percentile',percent_template],
                    'I': ['Price-to-Sales Ratio', float_template],
                    'J': ['PS Percentile', percent_template],
                    'K': ['EV/EBITDA', float_template],
                    'L': ['EV/EBITDA Percentile', percent_template],
                    'M': ['EV/GP', float_template],
                    'N': ['EV/GP Percentile', percent_template],
                    'O': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [56]:
writer.save()